# ニューラルネットワークと実装プログラム解説

ニューラルネットワークの初歩的解説です。ディープラーニングの理論に興味がある方は学習に挑戦してみてください。

## 勾配降下法

説明変数$x$から、目的変数$y$を予測したいとします。

これを非常にシンプルな線形な関数$f(x) = ax+b$でモデル化するとします。

仮に、$x$と$y$が$y = 2x+3$の関係であったとしましょう。

我々は本当の値$a=2, b=3$を知りません。しかし、観測された一部のサンプルが存在するとします。

In [ ]:
import numpy as np

def random():
    x = np.random.rand(50)
    noise = np.random.rand(50) / 10
    y = 2 * x + 3 + noise
    return zip(x,y)

# ランダムにノイズの乗った50個のサンプルを作成します
sample = random()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
sample_xs, sample_ys = np.asarray(sample).transpose()
plt.scatter(sample_xs, sample_ys, s=2)
plt.show()

今回の上記のランダムに生成したデータは、人の目で見れば直線が簡単に予測できますね。これを機械的に学習するのが機械学習です。

試しに、$a=0, b=0$で初期化して、一致するか試してみます。

In [ ]:
def f(a, b, x):
    return a * x + b

for x, y in sample[:10]:
    print('{} <==> {}'.format(f(0,0,x), y))

当然、一致しません。しかし、サンプルデータを基に、なんとか本当の$a,b$を学習してみましょう。

ここで、勾配降下法を用いてみましょう。

まず、サンプルデータに対する予測結果と、サンプルの答えがどの程度違ったかを定量的に表します。これを誤差と呼びます。

そして、各パラメータを少しずらした時に誤差が小さくなるように、各パラメータを更新します。

具体的には、各パラメータ$w_i$を、誤差$E$に対する偏微分値を引いた値で更新します。

$$ w_i \leftarrow w_i - \lambda{\partial E \over \partial w_i} $$

![](img/b.1.png)

誤差を2乗誤差${1 \over 2}(y - f(x))^2$とし、パラメータ$a,b$それぞれに対する偏微分を計算して値を更新します。

$a,b$の誤差に対する偏微分は以下のようになります。

$${\partial E \over \partial a}={\partial E \over \partial f}{\partial f \over \partial a}=(f(x)-y)x$$
$${\partial E \over \partial b}={\partial E \over \partial f}{\partial f \over \partial b}=f(x)-y$$

In [ ]:
def f(a, b, x):
    return a * x + b

def loss(y, pred):
    return ((y - pred) * (y - pred))/2

a = 0
b = 0
lr = 0.1 # 学習率
for i in range(10):
    print('a={} b={}'.format(a,b))
    for x, y in sample:
        pred = f(a,b,x)
        E = loss(y, pred) # 二乗誤差
        
        # 誤差が小さくなるよう、パラメータに対する偏微分値でパラメータを更新
        a = a - lr * ((pred - y)*x)
        b = b - lr * (pred - y)

In [ ]:
plt.scatter(sample_xs, sample_ys, s=2)
_xs = np.linspace(0,1,10)
_ys = [f(a,b,x) for x in _xs]
plt.plot(_xs, _ys, color='red')
plt.show()

おそらくほぼ$a=2,b=3$になったのではないかと思います。これが勾配降下法です。

## TensorFlowで例を実装する

一次関数の学習をTensorFlowで実装してみます。

In [ ]:
import tensorflow as tf
import numpy as np

with tf.Graph().as_default() as graph:
    zero = tf.constant(0., dtype=tf.float64)
    
    # 引数
    x = tf.placeholder(tf.float64, name='x') # 入力
    y = tf.placeholder(tf.float64, name='y') # 教師データ

    # パラメータ
    a = tf.get_variable('a', initializer=zero)
    b = tf.get_variable('b', initializer=zero)

    # モデル
    pred = tf.identity(a * x + b, name='pred') # 予測値f(x)

    # 二乗誤差
    loss = tf.div(tf.squared_difference(y, pred), 2, name='loss')

    # 勾配降下法による最適化
    lr = 0.1 # 学習率
    opt = tf.train.GradientDescentOptimizer(lr) # 勾配降下法
    fit = opt.minimize(loss)

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer()) # パラメータの初期化

    for i in range(10):
        for _x, _y in sample:
            # 最適化の実行
            sess.run([fit], feed_dict={
                x: _x,
                y: _y,
            })
        # パラメータの取得
        [_a, _b] = sess.run([a,b])
        print('a={}, b={}'.format(_a,_b))

同じようにほぼ$a=2,b=3$になります。

## ニューラルネットワーク

実際のところ、現実での問題は$ax+b$などという直線の簡単な数式では表せません。それこそ、サンプルデータを集めて人の目で見ても理解が難しいものもあります。こういったものを機械的に学習する方法の1つに、ニューラルネットワークをモデルに用いる方法があります。

ニューラルネットワークは、生物に見られる神経細胞(ニューロン)のネットワークを模した巨大な演算関数です。十分に深い(多層な)ニューラルネットワークはより複雑なあらゆる関数を近似可能であることが知られています。

1つの基本的なニューロンは、以下のような数式で表されます

$$ \phi(Wx+b) $$

入力が$x$です。$W$は重みパラメータ、$b$はバイアスパラメータです。$W$と$b$はいわゆる定数項です。$Wx+b$の部分は、$W$と$x$の行列積で、バイアスが加算されます。この部分が線形変換であるのに対し、$\phi$関数は非線形関数が用いられます。

複数のニューロンの集合を層(layer)とみなした場合も、$\phi(Wx+b)$で表されます。この場合、$W,x,b$は全てベクトルまたは行列です。

Doodleでは、以下のようなニューラルネットワークが記述されています。

```python
with tf.variable_scope('model', initializer=initializer):
    x = image
    x = tf.layers.conv2d(x, 32, 5, padding='SAME', activation=tf.nn.relu)
    x = tf.layers.max_pooling2d(x, 2, 2, padding='SAME')
    x = tf.layers.conv2d(x, 64, 5, padding='SAME', activation=tf.nn.relu)
    x = tf.layers.max_pooling2d(x, 2, 2, padding='SAME')
    x = tf.reshape(x, [-1,7*7*64])
    x = tf.layers.dense(x, 1024, activation=tf.nn.relu)
    x = tf.layers.dropout(x, rate=dropout_rate, training=is_training)
    x = tf.layers.dense(x, 10)
```

ここでは、幾つかの層が使われています。

### `tf.layers.dense`

これは、密結合層と呼ばれる、ニューラルネットワークで最も基本的なニューロンの集合です。これは、$\phi(xW+b)$です。

上記の`tf.layers.dense(x, 1024, activation=tf.nn.relu)`では、$W$が`[7*7*64, 1024]`の行列、$b$が`[1024]`のベクトル、$\phi=\text{relu}$という意味になります。$\text{relu}(x)=max(x,0)$です。

### `tf.layers.conv2d`

畳み込み層と呼ばれます。画像認識における最頻出関数です。重みと入力の行列積の代わりに畳み込み演算を行います。$\phi(\text{conv2d}(x,W)+b)$です。

![](img/b.2.png)

`tf.layers.conv2d(x, 32, 5, padding='SAME', activation=tf.nn.relu)`の場合、5x5のカーネルサイズの32枚のフィルタ(実際には、上記図ではカラーチャネルの次元が省略されており、実体は5x5x1x32の4次元テンソルで表される重みパラメータです)を、ストライド長(デフォルトの1)ずつずらしながら入力に対し内積の総和を計算していきます。

### `tf.layers.max_pooling2d`

プーリング層と呼ばれます。指定した範囲内(2)をストライド長(2)ずつずらしながらに、その中の最大値だけを選んでデータを集約します。画像のダウンサンプリングの効果があります。

![](img/b.3.png)

### `tf.layers.dropout`

ドロップアウトと呼ばれます。学習時にパラメータを`rate`の確率で無効化します。学習データの偏りによる過学習を抑える効果があります。

論文:

> Srivastava, Nitish, et al. "Dropout: A simple way to prevent neural networks from overfitting." The Journal of Machine Learning Research 15.1 (2014): 1929-1958.

### 誤差の定義

最初の勾配降下法の例と同様に誤差を定義します。Doodleでは`sparse_softmax_cross_entropy`という関数を用いています。

```python
with tf.variable_scope('losses'):
    # クロスエントロピーを計算して誤差に追加します
    cross_entropy_loss = tf.losses.sparse_softmax_cross_entropy(
        labels=labels, logits=logits)

    # モデルで追加された全ての誤差の総和を取得します
    total_loss = tf.losses.get_total_loss()
```

これは、ネットワークの出力を`softmax`関数で離散確率分布に変換し、教師データとの交差エントロピーを計算します。

`softmax`関数は、以下の式で表されます。

$$ \text{softmax}(x) = {exp(x) \over ∑_{j} exp(x_j)} $$

引数のベクトル`x`に指数関数を適用し、その総和で割ります。総和で割るということは、出力の総和は必ず1になります。そのため、モデルの出力を確率分布と見なして扱いたい場合に頻繁に用いられます。

交差エントロピーは、教師データ$y$と予測$y'$に対して以下の式で表されます。ただし、対数関数が使われているため、数式をそのまま実装すると$y'_i=0$の場合に計算不可能になってしまいます。実際には計算的に安定するように実装されている`sparse_softmax_cross_entropy`のような関数を使うのが望ましいでしょう。

$$ H_{y}(y') = -\sum_i y_i \log(y'_i) $$

教師データは、内部的にOne-Hot Vector(OHV)で扱われます。これは引数`labels`の表す数字をインデクスとしたときに、そのインデクスだけが1であるような確率分布です。以下に例を示します。

```
label=2:
[0,0,1,0,0,0,0,0,0,0]

label=9:
[0,0,0,0,0,0,0,0,0,1]

label=3:
[0,0,0,1,0,0,0,0,0,0]
```

OHVを確率分布と見なすと、そのインデクスである確率が100%の離散確率分布と捉えられます。モデルの出力である確率分布と、このOHV(答えの確率が100%になっている確率分布)との誤差を計算しています。

交差エントロピーは、2つの情報量がどれだけ異なっているのかを表す関数で、2つの確率分布が完全に一致する場合は0となり最小値をとります。2つの確率分布の値がかけ離れるほどに、値は大きくなります。すなわち最初の例のようにこの誤差を最小化するようにニューラルネットワークのパラメータを更新することでモデルを学習することができます。

### 最適化

誤差を最小化するようにパラメータを最適化するとき、TensorFlowでは自分で微分を計算する必要はありません。

```python
global_step = tf.train.get_or_create_global_step()
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.variable_scope('optimizer'), tf.control_dependencies(update_ops):
    # total_loss(誤差の総和)が小さくなるようにパラメータを更新します
    optimizer = tf.train.AdamOptimizer(learning_rate)
    fit = optimizer.minimize(total_loss, global_step)
```

`tf.train.◯◯Optimizer`というクラスを使うと、自動でバックプロパゲーションが行われ、`minimize`に指定した誤差が小さくなるように、定義した学習対象のパラメータを自動で更新してくれます。最初に用いた(最急)勾配降下法をベースとした、より発展的な様々なアルゴリズムがサポートされています。Doodleの例では、Adamというアルゴリズムを利用しています。

論文:

> KINGMA, Diederik P.; BA, Jimmy. Adam: A method for stochastic optimization. arXiv preprint arXiv:1412.6980, 2014.